In [1]:
import gc
import time
import numpy as np
import pandas as pd
from sklearn.cross_validation import train_test_split
import xgboost as xgb
from xgboost import plot_importance
import matplotlib.pyplot as plt

/home/ubuntu/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [9]:
%%time
merged_click_data = pd.read_feather("../data/merged_click_data")

CPU times: user 7.87 s, sys: 6.74 s, total: 14.6 s
Wall time: 14.6 s


In [10]:
merged_click_data.dtypes

app                                              uint16
channel                                          uint16
click_id                                         uint32
device                                           uint16
ip                                               uint32
is_attributed                                     uint8
os                                               uint16
click_count_by_ip_os_device_dow                  uint32
click_count_by_ip_os_device_dow_hour             uint16
click_count_by_ip                                uint32
dow                                               uint8
hour                                              uint8
minute                                            uint8
second                                            uint8
rank_by_ip                                       uint32
rank_by_ip_os_device                             uint32
rank_by_ip_os_device_dow                         uint32
click_time_interval_by_ip                       

In [11]:
columns = ['app', 'channel', 'device', 'os', 'hour', 'dow', 
           'click_time_interval_by_ip_os_device_dow_hour',
           'click_count_by_ip_os_device_dow_hour', 'is_attributed']

In [12]:
%%time
# merged_click_data.drop(columns=["click_id"], inplace=True)
merged_click_data = merged_click_data[columns]
train = merged_click_data[merged_click_data["dow"] < 3]
test = merged_click_data[merged_click_data["dow"]==3]

CPU times: user 9.62 s, sys: 4.23 s, total: 13.9 s
Wall time: 13.9 s


In [13]:
del merged_click_data
gc.collect()

53

In [14]:
%%time
y_train = train["is_attributed"].astype("int")
train.drop(columns=["is_attributed"], inplace=True)
y_valid = test["is_attributed"].astype("int")
test.drop(columns=["is_attributed"], inplace=True)
gc.collect()

CPU times: user 2.08 s, sys: 1.78 s, total: 3.87 s
Wall time: 3.87 s


In [15]:
dtrain = xgb.DMatrix(train, y_train)
dvalid = xgb.DMatrix(test, y_valid)

In [16]:
del train, y_train, test, y_valid
gc.collect()

0

In [17]:
params = {'eta': 0.6,
          'tree_method': "hist",
          'grow_policy': "lossguide",
          'max_leaves': 7,
          'max_depth': 3,
          'subsample': 0.9,
          'colsample_bytree': 0.7,
          'colsample_bylevel': 0.7,
          'min_child_weight': 0,
          'alpha': 4,
          'objective': 'binary:logistic',
          'scale_pos_weight': 9,
          'eval_metric': 'auc',
          'nthread': 40,
          'random_state': 99,
          'silent': True}
watchlist = [(dtrain, 'train'), (dvalid, 'valid')]
model = xgb.train(params, dtrain, 500, watchlist,
                  maximize=True, early_stopping_rounds=25, verbose_eval=5)

[0]	train-auc:0.846571	valid-auc:0.828064
Multiple eval metrics have been passed: 'valid-auc' will be used for early stopping.

Will train until valid-auc hasn't improved in 25 rounds.
[5]	train-auc:0.919132	valid-auc:0.909055
[10]	train-auc:0.949361	valid-auc:0.942632
[15]	train-auc:0.954476	valid-auc:0.946122
[20]	train-auc:0.959246	valid-auc:0.95344
[25]	train-auc:0.962135	valid-auc:0.956632
[30]	train-auc:0.962184	valid-auc:0.956297


KeyboardInterrupt: 